In [1]:
import optuna
import os
import trako as TKO
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
DATASETNAME = 'ADHD_STFC_diff_for_Daniel'
DATADIR = '/home/d/DATA/Dropbox/TRAKODATA/'+DATASETNAME+'/'
SUBJECTS = ['101','201']
GROUP = ['tracts_left_hemisphere']

In [3]:
import sys
sys.path.append('../trako')
import tuner

In [7]:
OUTPUTDIR = '/tmp/TRAKO_OPTUNA1/'
vtpsizes = []
tkosizes = []
for S in SUBJECTS[0:1]:
    for G in GROUP[0:1]:
        CURRENTDIR = os.path.join(DATADIR, S, G)
        CURRENTOUTDIR = os.path.join(OUTPUTDIR, S, G)
        files = os.listdir(CURRENTDIR)[0:1]
        
        if not os.path.exists(CURRENTOUTDIR):
            os.makedirs(CURRENTOUTDIR)
        
        for f in files:
            if not f.endswith('vtp'):
                continue
            
            vtpfile = os.path.join(CURRENTDIR, f)
            
            tuner.Tuner.tune(vtpfile, 0.005)

VTP size 1172155
TKO default size 377009


[I 2020-01-30 09:52:59,278] Setting status of trial#0 as TrialState.PRUNED. 
[I 2020-01-30 09:53:00,704] Setting status of trial#1 as TrialState.PRUNED. 
[I 2020-01-30 09:53:01,042] Setting status of trial#2 as TrialState.PRUNED. 
[I 2020-01-30 09:53:01,415] Setting status of trial#3 as TrialState.PRUNED. 
[I 2020-01-30 09:53:03,612] Setting status of trial#4 as TrialState.PRUNED. 
[I 2020-01-30 09:53:05,025] Setting status of trial#5 as TrialState.PRUNED. 
[I 2020-01-30 09:53:05,355] Setting status of trial#6 as TrialState.PRUNED. 
[I 2020-01-30 09:53:05,703] Setting status of trial#7 as TrialState.PRUNED. 
[I 2020-01-30 09:53:12,983] Finished trial#8 resulted in value: 1013359.0. Current best value is 1013359.0 with parameters: {'position': False, 'sequential': False, 'quantization_bits': 26, 'compression_level': 0}.
[I 2020-01-30 09:53:13,209] Setting status of trial#9 as TrialState.PRUNED. 
[I 2020-01-30 09:53:14,710] Setting status of trial#10 as TrialState.PRUNED. 
[I 2020-01-30 

[I 2020-01-30 09:55:44,335] Finished trial#65 resulted in value: 786391.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:55:46,338] Setting status of trial#66 as TrialState.PRUNED. 
[I 2020-01-30 09:55:50,100] Finished trial#67 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:55:52,098] Setting status of trial#68 as TrialState.PRUNED. 
[I 2020-01-30 09:55:55,978] Finished trial#69 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:55:56,259] Setting status of trial#70 as TrialState.PRUNED. 
[I 2020-01-30 09:56:00,071] Finished trial#71 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'po

[I 2020-01-30 09:58:18,273] Finished trial#117 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:58:22,043] Finished trial#118 resulted in value: 695199.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:58:25,746] Finished trial#119 resulted in value: 740855.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:58:27,757] Setting status of trial#120 as TrialState.PRUNED. 
[I 2020-01-30 09:58:31,606] Finished trial#121 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 09:58:33,651] Setting status of trial#122 as TrialS

[I 2020-01-30 10:00:51,219] Finished trial#166 resulted in value: 695199.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:00:53,273] Setting status of trial#167 as TrialState.PRUNED. 
[I 2020-01-30 10:00:57,110] Finished trial#168 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:00:59,210] Setting status of trial#169 as TrialState.PRUNED. 
[I 2020-01-30 10:01:03,073] Finished trial#170 resulted in value: 695199.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:01:06,841] Finished trial#171 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compressio

[I 2020-01-30 10:03:25,426] Setting status of trial#213 as TrialState.PRUNED. 
[I 2020-01-30 10:03:29,442] Finished trial#214 resulted in value: 695199.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:03:33,326] Finished trial#215 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:03:35,511] Setting status of trial#216 as TrialState.PRUNED. 
[I 2020-01-30 10:03:39,584] Finished trial#217 resulted in value: 695199.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:03:43,494] Finished trial#218 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compressio

[I 2020-01-30 10:06:04,971] Finished trial#259 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:06:08,912] Finished trial#260 resulted in value: 740855.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:06:12,815] Finished trial#261 resulted in value: 649395.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 2}.
[I 2020-01-30 10:06:15,006] Setting status of trial#262 as TrialState.PRUNED. 
[I 2020-01-30 10:06:17,419] Setting status of trial#263 as TrialState.PRUNED. 
[I 2020-01-30 10:06:21,462] Finished trial#264 resulted in value: 695199.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compressio

KeyboardInterrupt: 

In [15]:
study.best_trial

FrozenTrial(number=6, state=TrialState.COMPLETE, value=111336.0, datetime_start=datetime.datetime(2020, 1, 30, 8, 50, 20, 536952), datetime_complete=datetime.datetime(2020, 1, 30, 8, 50, 20, 921601), params={'position': False, 'sequential': True, 'quantization_bits': 6, 'compression_level': 0}, distributions={'position': CategoricalDistribution(choices=(True, False)), 'sequential': CategoricalDistribution(choices=(True, False)), 'quantization_bits': IntUniformDistribution(high=31, low=0), 'compression_level': IntUniformDistribution(high=10, low=0)}, user_attrs={}, system_attrs={'_number': 6}, intermediate_values={}, trial_id=6)

In [16]:
study.best_trial.params

{'position': False,
 'sequential': True,
 'quantization_bits': 6,
 'compression_level': 0}

In [18]:
import imp, os
tkompare = imp.load_source('tkompare', '../tkompare')

config = {
  '*': { 
    'position':study.best_trial.params['position'],
    'sequential':study.best_trial.params['sequential'],
    'quantization_bits':study.best_trial.params['quantization_bits'],
    'compression_level':study.best_trial.params['compression_level'],
    'quantization_range':-1,
    'quantization_origin':None
  }

}



tkompare.compare(vtpfile, '/tmp/restored.vtp')

+-------------------+---------+-------------+
|        File       |  Bytes  | Streamlines |
+-------------------+---------+-------------+
| cluster_00616.vtp | 1172155 |     190     |
|    restored.vtp   |  974914 |     190     |
+-------------------+---------+-------------+
+--------------+------------+------------+-------------+----------+
|   Geometry   | Min. Delta | Max. Delta | Mean. Delta | StdDev.  |
+--------------+------------+------------+-------------+----------+
|  XYZ Points  |  36.01881  | 121.135544 |   66.63499  | 14.55739 |
| Line Indices |    0.0     |    0.0     |     0.0     |   0.0    |
+--------------+------------+------------+-------------+----------+
+---------------------------------+------------+------------+-------------+-----------+
|             Scalars             | Min. Delta | Max. Delta | Mean. Delta |  StdDev.  |
+---------------------------------+------------+------------+-------------+-----------+
| NormalizedSignalEstimationError |    0.0     |  0.